## Load, enrich, augment and clean the data

In [1]:
# dataset is taken from Kaggle and downloaded from here:
# https://www.kaggle.com/datasets/thomasnibb/amsterdam-house-price-prediction
# license is CC0: Public Domain - can copy, modify, distribute and perform the work, 
# even for commercial purposes, all without asking permission.

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv("HousingPrices-Amsterdam-August-2021.csv")
df.head() # look of the original dataset


,Unnamed: 0,Address,Zip,Price,Area,Room,Lon,Lat
0,1,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3,4.907736,52.356157
1,2,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3,4.850476,52.348586
2,3,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4,4.944774,52.343782
3,4,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6,4.789928,52.343712
4,5,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5,4.902503,52.410538


In [2]:
# STEP 1 - make columns look good:
df.columns = df.columns.str.lower()

# We will use only 4 columns for the start
used_cols = [ 'zip', 'address', 'area', 'room', 'price']
df = df[used_cols]
df.head(10)

,zip,address,area,room,price
0,1091 CR,"Blasiusstraat 8 2, Amsterdam",64,3,685000.0
1,1059 EL,"Kromme Leimuidenstraat 13 H, Amsterdam",60,3,475000.0
2,1097 SM,"Zaaiersweg 11 A, Amsterdam",109,4,850000.0
3,1060 TH,"Tenerifestraat 40, Amsterdam",128,6,580000.0
4,1036 KN,"Winterjanpad 21, Amsterdam",138,5,720000.0
5,1051 AM,"De Wittenkade 134 I, Amsterdam",53,2,450000.0
6,1033 KM,"Pruimenstraat 18 B, Amsterdam",87,3,450000.0
7,1053 WL,"Da Costakade 32 II, Amsterdam",80,2,590000.0
8,1058 DG,"Postjeskade 41 2, Amsterdam",49,3,399000.0
9,1073 TM,"Van Ostadestraat 193 H, Amsterdam",33,2,300000.0


In [3]:
df.describe()

,area,room,price
count,924.000000,924.000000,9.200000e+02
mean,95.952381,3.571429,6.220654e+05
std,57.447436,1.592332,5.389942e+05
min,21.000000,1.000000,1.750000e+05
25%,60.750000,3.000000,3.500000e+05
50%,83.000000,3.000000,4.670000e+05
75%,113.000000,4.000000,7.000000e+05
max,623.000000,14.000000,5.950000e+06


In [4]:
df.zip.nunique() # 834 - too many for grouping...

834

In [5]:
df.address.nunique() # 919 - too many as well...

919

In [6]:
# idea - we need to extract city districts (wijk in Dutch) based on the zip/street address
# collect a sentiment score for each district in range 1-10 using LLM - good or bad for living
# and add it as columns to our dataset - district and sentiment - as realtors often say - location, location, location...

# I decided to drop GPS lat long columns - not to mess up with Google maps API for simplicity
# and used Amsterdam district names from here:
# https://data.europa.eu/data/datasets/22433-postcode-buurt-wijk?locale=da

# idea 2 - in Amsterdam city district - wijk consists of subdistricts - buurt
# which consists of PC4 post codes (first 4 digits of zip column in our dataset)
# I will drop the street address for now and map buurt and wijk from downloaded CSV to PC4

In [7]:
# STEP 2 - extract city districts (wijk in Dutch) and enrich our primary dataset by district mappings

mappings = pd.read_csv('postcode-buurt-wijk.csv', sep=';', engine='python')
mappings.head()

,objectid,straatnaam,postcode6,postcode4,hsnr_laag,hsnr_hoog,hsnr_aant,buurtcode,buurtnaam,wijkcode,wijknaam,stadsdeelcode,stadsdeelnaam,x_rd,y_rd,longitude,latitude,geo_shape,geo_point_2d
0,576,Bordeauxlaan,5627GV,5627,2.0,52.0,26.0,522,Achtse Barrier-Gunterslaer,52,Wijk Achtse Molen,5,Stadsdeel Woensel-Noord,159219.520623,387947.444850,5.447943,51.480558,"{""coordinates"": [5.447941554005463, 51.4805580...","51.480558093244355, 5.447941554005463"
1,577,Bordeauxlaan,5627GW,5627,54.0,114.0,31.0,522,Achtse Barrier-Gunterslaer,52,Wijk Achtse Molen,5,Stadsdeel Woensel-Noord,159175.302700,387933.685308,5.447306,51.480435,"{""coordinates"": [5.447304851676096, 51.4804347...","51.4804347440077, 5.447304851676096"
2,582,Borodinlaan,5653HL,5653,1.0,13.0,7.0,721,Genderdal,72,Wijk Oud-Gestel,7,Stadsdeel Gestel,159288.747017,381420.180850,5.448860,51.421887,"{""coordinates"": [5.448858744580933, 51.4218862...","51.421886226518914, 5.448858744580933"
3,583,Borretpad,5652GD,5652,2.0,24.0,12.0,622,Het Ven,62,Wijk Halve Maan,6,Stadsdeel Strijp,159076.165439,383407.931050,5.445827,51.439755,"{""coordinates"": [5.445825412686405, 51.4397550...","51.43975504310988, 5.445825412686405"
4,586,Bosboomstraat,5613KE,5613,2.0,36.0,17.0,312,Lakerlopen,31,Wijk De Laak,3,Stadsdeel Tongelre,162842.491788,383522.568500,5.499999,51.440746,"{""coordinates"": [5.499997589329138, 51.4407455...","51.44074558984123, 5.499997589329138"


In [8]:
# small explanation how Dutch postal addresses are organised
# The PC4 (first four digits of zip code) identifies a broad area within a city — 
# this can be mapped to neighbourhood or district statistically.
# The PC6 (full 6 characters of zip code, including letters) often maps to one side 
# of a street, or even a cluster of house numbers.
# The letter pair (e.g., CR in zip = '1091 CR') is simply a unique subdivision to make post sorting precise.

# Example:
# full zip  = 1091 CR
# street address: Blasiusstraat 8 2, Amsterdam
# Neighbourhood (Buurt): Weesperzijde
# District (Wijk): Oud-Oost
# Part of the city (Stadsdeel): Amsterdam-Oost

# so the naming hierachy is postcode → buurt → wijk


# prepare our df zip code col for mapping
df['postcode6'] = df.zip.str.replace(" ", "")
df.head()

,zip,address,area,room,price,postcode6
0,1091 CR,"Blasiusstraat 8 2, Amsterdam",64,3,685000.0,1091CR
1,1059 EL,"Kromme Leimuidenstraat 13 H, Amsterdam",60,3,475000.0,1059EL
2,1097 SM,"Zaaiersweg 11 A, Amsterdam",109,4,850000.0,1097SM
3,1060 TH,"Tenerifestraat 40, Amsterdam",128,6,580000.0,1060TH
4,1036 KN,"Winterjanpad 21, Amsterdam",138,5,720000.0,1036KN


In [12]:
# clean up our mappings dataframe and prepare for merging
mappings = mappings[['straatnaam', 'postcode6', 'buurtcode', 'buurtnaam', 'wijkcode', 'wijknaam']]
mappings.head(1)


,straatnaam,postcode6,buurtcode,buurtnaam,wijkcode,wijknaam
0,Bordeauxlaan,5627GV,522,Achtse Barrier-Gunterslaer,52,Wijk Achtse Molen


In [15]:
# merge 2 dataftames
# df = df.merge(mappings, on='postcode6')
# df.head(1)
df_merged = df.merge(mappings, on="postcode6", how="left")
df_merged.head(1)

,zip,address,area,room,price,postcode6,straatnaam,buurtcode,buurtnaam,wijkcode,wijknaam
0,1091 CR,"Blasiusstraat 8 2, Amsterdam",64,3,685000.0,1091CR,NaN,NaN,NaN,NaN,NaN


In [16]:
print(df['postcode6'].head(10).tolist())
print(mappings['postcode6'].head(10).tolist())


['1091CR', '1059EL', '1097SM', '1060TH', '1036KN', '1051AM', '1033KM', '1053WL', '1058DG', '1073TM']
['5627GV', '5627GW', '5653HL', '5652GD', '5613KE', '5612AM', '5612HC', '5612HG', '5621JA', '5621JB']


In [17]:
set(df['postcode6']).intersection(set(mappings['postcode6']))


set()